In [6]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import pandas as pd
import numpy as np
from pathlib import Path

In [7]:
# ["louisville", "san_jose", "columbia", "north_birmingham", "jackson_county", "duluth", "hill_district"]
community = 'duluth' # TODO change community name
fig_dir = Path(".").resolve() / "results" / community
fig_dir.mkdir(exist_ok=True, parents=True)

community_results = pd.read_parquet(f"data_/community_building_samples_with_upgrade_cost_and_bill/{community}/processed_upgrade_results.parquet")

### get energy burden distribution

In [8]:
up00 = community_results.loc[community_results["upgrade_name"]=="baseline"]
metric = "energy_burden"
metric_name = metric.replace("_", " ").title()

cooling_type_list = ['<2%', '2-4%', '4-6%', '6-8%', '8-10%', '10%+']
cooling_type_percentage = []
for cooling_type in cooling_type_list:
    temp = up00.loc[up00[metric] == cooling_type]
    temp_count =temp['sample_weight'].sum()/up00["sample_weight"].sum()
    cooling_type_percentage.append(temp_count)
    
fig = px.bar(x=cooling_type_list, y=cooling_type_percentage, text_auto=True)
fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
fig.update_layout(autosize=False, width=350, height=300, 
                  font=dict(size=10), 
                  plot_bgcolor='rgba(0, 0, 0, 0)', 
                  title=metric_name,
                  xaxis_title="",
                  yaxis_title="Percent of Housing",
                  yaxis_range=[0,0.3])
fig.layout.yaxis.tickformat = ',.0%'
fig.update_xaxes(tickangle=45)
fig.show()
fig.write_image(fig_dir / f"{metric}.svg")

# Add segment label to the results

Define segement

In [9]:
####for columbia
###### fuel
# 'electricity', 'natural_gas', 'fuel_oil', 'propane'
###### Vacancy Status
# 'Occupied', 'Vacant'

###### Building Type
# 'Mobile Home'
# 'Multi-Family with 2 - 4 Units'
# 'Multi-Family with 5+ Units'
# 'Single-Family Attached'
# 'Single-Family Detached'
###### Vintage
# '<1940', '1940s', '1950s', '1960s', '1970s', '1980s', '1990s', '2000s', '2010s'
###### Tenure
# 'Renter', 'Owner'
###### area_median_income
# '0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%'   build_existing_model.area_median_income

######build_existing_model.geometry_wall_exterior_finish
#'Brick', 'Concrete', 'Steel Frame', 'Wood Frame'

#####TODO, change the community needs follwing
'''
Single family detached, 1940-1979, all income 
Multifamily with 5+ units, 1940-1979, all income 
Single family detached, 1980+, all income 
Multifamily with 5+ units, 1980+, all income 
Single family detached, 1940-1979, 0-80% AMI 
Multifamily with 5+ units, 1940-1979, 0-80% AMI 
Single family detached, 1980+, 0-80% AMI 
Multifamily with 5+ units, 1980+, 0-80% AMI 
Only interested in electric and natural gas fuel types 
'''

seg1_building_type = ['Single-Family Detached']
seg1_vintage = ['1940s', '1950s', '1960s', '1970s']
seg1_tenure = ['Renter', 'Owner']
seg1_construction = ['Brick', 'Concrete', 'Steel Frame', 'Wood Frame']
seg1_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

seg2_building_type = ['Multi-Family with 5+ Units']
seg2_vintage = ['1940s', '1950s', '1960s', '1970s']
seg2_tenure = ['Renter', 'Owner']
seg2_construction = ['Brick', 'Concrete', 'Steel Frame', 'Wood Frame']
seg2_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

seg3_building_type = ['Single-Family Detached']
seg3_vintage = ['1980s', '1990s', '2000s', '2010s']
seg3_tenure = ['Renter','Owner']
seg3_construction = ['Brick', 'Concrete', 'Steel Frame', 'Wood Frame']
seg3_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

seg4_building_type = ['Multi-Family with 5+ Units']
seg4_vintage = ['1980s', '1990s', '2000s', '2010s']
seg4_tenure = ['Renter','Owner']
seg4_construction = ['Brick', 'Concrete', 'Steel Frame', 'Wood Frame']
seg4_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

In [10]:
####for san_jose
'''
Multifamily with 5+ units, 1940-1979, all income 
Multifamily with 5+ units, 1980+, all income 
Single family detached 1940-1979, all income 
Single family attached 1940-1979, all income 
Multifamily with 5+ units, 1940-1979, 0-80% AMI 
Multifamily with 5+ units, 1980+, 0-80% AMI 
Single family detached 1940-1979, 0-80% AMI 
Single family attached 1940-1979, 0-80% AMI 
Only interested in electricity and natural gas fuel type 
Do not want results for vacant housing  
'''
vacancy_status = ['Occupied']

seg1_building_type = ['Multi-Family with 5+ Units']
seg1_vintage = ['1940s', '1950s', '1960s', '1970s']
seg1_tenure = ['Renter', 'Owner']
seg1_construction = ['Brick', 'Concrete', 'Steel Frame', 'Wood Frame']
seg1_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

seg2_building_type = ['Multi-Family with 5+ Units']
seg2_vintage = ['1980s', '1990s', '2000s', '2010s']
seg2_tenure = ['Renter', 'Owner']
seg2_construction = ['Brick', 'Concrete', 'Steel Frame', 'Wood Frame']
seg2_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

seg3_building_type = ['Single-Family Detached']
seg3_vintage = ['1940s', '1950s', '1960s', '1970s']
seg3_tenure = ['Renter','Owner']
seg3_construction = ['Brick', 'Concrete', 'Steel Frame', 'Wood Frame']
seg3_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

seg4_building_type = ['Single-Family Attached']
seg4_vintage = ['1940s', '1950s', '1960s', '1970s']
seg4_tenure = ['Renter','Owner']
seg4_construction = ['Brick', 'Concrete', 'Steel Frame', 'Wood Frame']
seg4_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

In [11]:
####for duluth
'''
Single family detached, pre 1940, renter, all income 
Single family detached, pre 1940, owner, all income 
Multifamily with 2-4 units, 1940-1979, all income 
Multifamily with 5+ units, 1940-1979, all income 
Single family detached, pre 1940, renter, 0-80% AMI 
Single family detached, pre 1940, owner, 0-80% AMI 
Multifamily with 2-4 units, 1940-1979, 0-80% AMI 
Multifamily with 5+ units, 1940-1979, 0-80% AMI 
Only interested in electricity, natural gas, and fuel oil fuel types 
Do not want vacant housing g  
'''

vacancy_status = ['Occupied']

seg1_building_type = ['Single-Family Detached']
seg1_vintage = ['<1940']
seg1_tenure = ['Renter']
seg1_construction = ['Brick', 'Concrete', 'Steel Frame', 'Wood Frame']
seg1_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

seg2_building_type = ['Single-Family Detached']
seg2_vintage = ['<1940']
seg2_tenure = ['Owner']
seg2_construction = ['Brick', 'Concrete', 'Steel Frame', 'Wood Frame']
seg2_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

seg3_building_type = ['Multi-Family with 2 - 4 Units']
seg3_vintage = ['1940s', '1950s', '1960s', '1970s']
seg3_tenure = ['Renter','Owner']
seg3_construction = ['Brick', 'Concrete', 'Steel Frame', 'Wood Frame']
seg3_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

seg4_building_type = ['Multi-Family with 5+ Units']
seg4_vintage = ['1940s', '1950s', '1960s', '1970s']
seg4_tenure = ['Renter','Owner']
seg4_construction = ['Brick', 'Concrete', 'Steel Frame', 'Wood Frame']
seg4_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

In [12]:
####for louisville
'''
Multifamily with 2-4 units, pre 1940, 0-80% AMI 
Multifamily with 2-4 units, 1940-1979, 80-150% AMI 
Multifamily with 5+ units, pre 1940, 0-80% AMI 
Multifamily with 5+ units, 1940-1979, 80-150% AMI 
Multifamily with 2-4 units, pre 1940, all income 
Multifamily with 2-4 units, 1940-1979, all income 
Multifamily with 5+ units, pre 1940, all income 
Multifamily with 5+ units, 1940-1979, all income 
Do not want ‘Other Fuels’ as a fuel type 
Do not want vacant housing 
'''
vacancy_status = ['Occupied']

seg1_building_type = ['Multi-Family with 2 - 4 Units']
seg1_vintage = ['<1940']
seg1_tenure = ['Renter','Owner']
seg1_construction = ['Brick', 'Concrete', 'Steel Frame', 'Wood Frame']
seg1_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

seg2_building_type = ['Multi-Family with 2 - 4 Units']
seg2_vintage = ['1940s', '1950s', '1960s', '1970s']
seg2_tenure = ['Renter','Owner']
seg2_construction = ['Brick', 'Concrete', 'Steel Frame', 'Wood Frame']
seg2_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

seg3_building_type = ['Multi-Family with 5+ Units']
seg3_vintage = ['<1940']
seg3_tenure = ['Renter','Owner']
seg3_construction = ['Brick', 'Concrete', 'Steel Frame', 'Wood Frame']
seg3_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

seg4_building_type = ['Multi-Family with 5+ Units']
seg4_vintage = ['1940s', '1950s', '1960s', '1970s']
seg4_tenure = ['Renter','Owner']
seg4_construction = ['Brick', 'Concrete', 'Steel Frame', 'Wood Frame']
seg4_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

In [13]:
####for hill_district
'''
Multifamily with 2-4 units, 1940-1979, all income 
Multifamily with 5+ units, 1980+, all income 
Single family detached, pre 1940, masonry construction, all income 
Single family detached, 1940-1979, masonry construction, all income 
Multifamily with 2-4 units, 1940-1979, 0-80% AMI 
Multifamily with 5+ units, 1980+, 0-80% AMI
Single family detached, pre 1940, masonry construction, 0-80% AMI 
Single family detached, 1940-1979, masonry construction, 0-80% AMI 
Only interested in electricity and natural gas fuel type 
'''
vacancy_status = ['Occupied']

seg1_building_type = ['Multi-Family with 2 - 4 Units']
seg1_vintage = ['1940s', '1950s', '1960s', '1970s']
seg1_tenure = ['Renter','Owner']
seg1_construction = ['Brick', 'Concrete', 'Steel Frame', 'Wood Frame']
seg1_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

seg2_building_type = ['Multi-Family with 5+ Units']
seg2_vintage = ['1980s', '1990s', '2000s', '2010s']
seg2_tenure = ['Renter','Owner']
seg2_construction = ['Brick', 'Concrete', 'Steel Frame', 'Wood Frame']
seg2_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

seg3_building_type = ['Single-Family Detached']
seg3_vintage = ['<1940']
seg3_tenure = ['Renter','Owner']
seg3_construction = ['Brick']
seg3_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

seg4_building_type = ['Single-Family Detached']
seg4_vintage = ['1940s', '1950s', '1960s', '1970s']
seg4_tenure = ['Renter','Owner']
seg4_construction = ['Brick']
seg4_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

label each building sample

In [14]:
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg1_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg1_vintage) & 
                     community_results['build_existing_model.tenure'].isin(seg1_tenure) &  
                     community_results['build_existing_model.geometry_wall_type'].isin(seg1_construction) & 
                     community_results['build_existing_model.area_median_income'].isin(seg1_median_income),
                     'segment'] = 'segment1'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg2_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg2_vintage) & 
                     community_results['build_existing_model.tenure'].isin(seg2_tenure) &  
                     community_results['build_existing_model.geometry_wall_type'].isin(seg2_construction) & 
                     community_results['build_existing_model.area_median_income'].isin(seg2_median_income),
                     'segment'] = 'segment2'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg3_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg3_vintage) & 
                     community_results['build_existing_model.tenure'].isin(seg3_tenure) &   
                     community_results['build_existing_model.geometry_wall_type'].isin(seg3_construction) &
                     community_results['build_existing_model.area_median_income'].isin(seg3_median_income),
                     'segment'] = 'segment3'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg4_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg4_vintage) & 
                     community_results['build_existing_model.tenure'].isin(seg4_tenure) &   
                     community_results['build_existing_model.geometry_wall_type'].isin(seg4_construction) &
                     community_results['build_existing_model.area_median_income'].isin(seg4_median_income),
                     'segment'] = 'segment4'

### Get summary (including weighted percentiles)

In [15]:
def weighted_mean(x, metric: str):
    if len(x[metric].dropna()) == 0:
        res = 0
    else:
        res = (x[metric] * x["sample_weight"]).sum()/x["sample_weight"].sum()
    col_name = f"{metric} wt_mean"
    return pd.Series({col_name: res})

def weighted_percentile(x, metric: str, percents: int):
    col_name = f"{metric} wt_p{percents}"
    res = _weighted_percentile(x[metric], percents, weights=x["sample_weight"])
    return pd.Series({col_name: res})

def _weighted_percentile(data, percents: int, weights=None):
    """
    perc : percentile in 0 - 1
    """
    # dropna
    idx = ~np.isnan(data) & ~np.isnan(weights)
    data = np.array(data)[idx]
    if len(data) == 0:
        return np.nan
    if weights is None:
        return np.percentile(data, percents)
    weights = np.array(weights)[idx]
    ix = np.argsort(data)
    data = data[ix] # sort data
    weights = weights[ix] # sort weights
    cdf = (np.cumsum(weights) - 0.5 * weights) / np.sum(weights) # 'like' a CDF function
    return np.interp(percents, cdf, data)

def get_summary(df, metric, lower_percentile=0.25, upper_percentile=0.75):
    dfgb = df.groupby(["upgrade_name"])
    dfs = pd.concat([
        dfgb["sample_weight"].count().rename("n_samples"),
        dfgb["sample_weight"].sum().rename("n_represented"),
        dfgb.apply(lambda x: weighted_mean(x, metric)),
        dfgb.apply(lambda x: weighted_percentile(x, metric, percents=lower_percentile)),
        dfgb.apply(lambda x: weighted_percentile(x, metric, percents=upper_percentile)),
    ], axis=1)
    dfs = dfs.sort_values(by=f"{metric} wt_mean", ascending=False)

    return dfs

In [16]:
metric = 'pct_saving_energy.total_%'  # TODO 'pct_saving_cbill.total_%' #'pct_saving_energy.total_%' #'saving_energy.total_mmbtu' #'saving_cbill.total_usd'
segment = "segment3" #TODO (can put in a for-loop)

df_seg = community_results.loc[community_results["segment"]==segment]
get_summary(df_seg, metric, lower_percentile=0.25, upper_percentile=0.75)

,n_samples,n_represented,pct_saving_energy.total_% wt_mean,pct_saving_energy.total_% wt_p0.25,pct_saving_energy.total_% wt_p0.75
upgrade_name,,,,,
Basic Enclosure + High Efficiency Whole Home Electrification,1,48.021739,59.60,59.60,59.60
Enhanced Enclosure + High Efficiency Whole Home Electrification,1,48.021739,59.60,59.60,59.60
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,1,48.021739,58.92,58.92,58.92
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,1,48.021739,58.92,58.92,58.92
High Efficiency Whole Home Electrification,1,48.021739,50.56,50.56,50.56
High Efficiency Heat Pump with Electric Heat Backup,1,48.021739,48.41,48.41,48.41
Mininum Efficiency Whole Home Electrification,1,48.021739,33.55,33.55,33.55
Mininum Efficiency Heat Pump with Electric Heat Backup,1,48.021739,33.25,33.25,33.25
Basic Enclosure,1,48.021739,22.43,22.43,22.43


# 1. Whole Community

## 1.1 Upgrade Cost Per Dwelling Unit

In [17]:
def figure_mean_upgrade_cost(commnity_results):
    results = commnity_results[['upgrade_name',
                       'upgrade_cost_usd',
                                'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'upgrade_cost_usd': 'sum',
                    'sample_weight': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']/results['sample_weight']
    results = results.sort_values('upgrade_cost_usd')
    results['upgrade_cost_usd'] = results['upgrade_cost_usd'].round(1) 
    
    print('upgrade_cost_usd per dwelling unit')
    print(results['upgrade_cost_usd'])

figure_mean_upgrade_cost(community_results) 

upgrade_cost_usd per dwelling unit
upgrade_name
Electric Cooking                                                    1127.3
Electric Clothes Dryer                                              1266.0
Induction Cooking                                                   1953.1
Heat Pump Clothes Dryer                                             2547.5
Basic Enclosure                                                     2940.3
Heat Pump Water Heater                                              3194.7
Enhanced Enclosure                                                  4934.1
High Efficiency Heat Pump with Electric Heat Backup                31202.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        34316.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     34669.7
Basic Enclosure + High Efficiency Whole Home Electrification       38326.7
High Efficiency Whole Home Electrification                         38404.3
Enhanced Enclosure + High Efficiency Whole Home Elec

## 1.2 Total Annual Savings

In [18]:
# flavor 1 - with electricity and non-electricity split
def figure_total_savings(commnity_results):
    results = commnity_results[['upgrade_name',
                       'saving_energy.electricity_kwh',
                      'saving_energy.natural_gas_therm',
                                'saving_energy.fuel_oil_mmbtu',
                                'saving_energy.propane_mmbtu',
                       'saving_cbill.total_usd',
                       'saving_emission.total_kgCO2e',
                                'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_therm']*0.1 + results['saving_energy.fuel_oil_mmbtu']+ results['saving_energy.propane_mmbtu']
    results['saving_energy.electricity_kwh'] = results['saving_energy.electricity_kwh']*results['sample_weight']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_fuel_oil_propane_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.electricity_kwh': 'sum',
                     'saving_energy.natural_gas_fuel_oil_propane_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results = results.sort_values('saving_cbill.total_usd')
    
    fig = make_subplots(rows=1, cols=5, horizontal_spacing=0.12, column_widths=[0.001, 1, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.electricity_kwh'],
        y=results['upgrade_name'],
        text=results['saving_energy.electricity_kwh'],
        orientation='h'),row=1, col=4)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        orientation='h'),row=1, col=5)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=5)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill<br>Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Emission<br>Savings<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Electricity<br>Savings<br>kwh", xaxis4_visible=True, yaxis4_visible=False,
                      xaxis5_title="Gas, Propane,<br>Fuel Oil Savings<br>mmbtu", xaxis5_visible=True, yaxis5_visible=False,
                     title='Total Annual Savings for the Whole Community')
    fig.update_layout(
    autosize=False,
    width=900,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'total_annual_savings_1.svg')

figure_total_savings(community_results)

In [19]:
# flavor 2 - site, electricity, and non-electricity
def figure_total_savings(commnity_results):
    results = commnity_results[[
        'upgrade_name',
        'saving_energy.total_mmbtu',
        'saving_energy.electricity_kwh',
        'saving_energy.natural_gas_therm',
        'saving_energy.fuel_oil_mmbtu',
        'saving_energy.propane_mmbtu',
        'saving_cbill.total_usd',
        'saving_emission.total_kgCO2e',
        'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_therm']*0.1 + results['saving_energy.fuel_oil_mmbtu']+ results['saving_energy.propane_mmbtu']
    results['saving_energy.electricity_kwh'] = results['saving_energy.electricity_kwh']*results['sample_weight']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_fuel_oil_propane_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.electricity_kwh': 'sum',
                     'saving_energy.natural_gas_fuel_oil_propane_mmbtu': 'sum', 
                      'saving_energy.total_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results = results.sort_values('saving_cbill.total_usd')
    
    fig = make_subplots(rows=1, cols=6, horizontal_spacing=0.12, column_widths=[0.001, 1, 1, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)

    fig.add_trace(go.Bar(
        x=results['saving_energy.total_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.total_mmbtu'],
        orientation='h'),row=1, col=4)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.electricity_kwh'],
        y=results['upgrade_name'],
        text=results['saving_energy.electricity_kwh'],
        orientation='h'),row=1, col=5)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        orientation='h'),row=1, col=6)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=5)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=6)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill<br>Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Site Emission<br>Saving<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Site Energy<br>Saving<br>mmbtu", xaxis4_visible=True, yaxis4_visible=False,
                      xaxis5_title="Electricity<br>Saving<br>kwh", xaxis5_visible=True, yaxis5_visible=False,
                      xaxis6_title="Gas, Propane,<br>Fuel Oil Saving<br>mmbtu", xaxis6_visible=True, yaxis6_visible=False,
                     title='Community Total Annual Savings')
    fig.update_layout(
    autosize=False,
    width=1100,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'total_annual_savings_2.svg')

figure_total_savings(community_results)

In [20]:
# flavor 3 - site energy only
def figure_total_savings(commnity_results):
    results = commnity_results[[
        'upgrade_name',
        'saving_energy.total_mmbtu',
        'saving_cbill.total_usd',
        'saving_emission.total_kgCO2e',
        'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.total_mmbtu'] = results['saving_energy.total_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.total_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results = results.sort_values('saving_cbill.total_usd')
    
    fig = make_subplots(rows=1, cols=4, horizontal_spacing=0.1, column_widths=[0.02, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.total_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.total_mmbtu'],
        orientation='h'),row=1, col=4)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Emission Savings<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Energy Savings<br>mmbtu", xaxis4_visible=True, yaxis4_visible=False,
                     title='Total Annual Savings for the Whole Community')
    fig.update_layout(
    autosize=False,
    width=850,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'total_annual_savings_3.svg')

figure_total_savings(community_results) 

In [21]:
def figure_mean_savings(commnity_results):
    results = commnity_results[['upgrade_name',
                       'saving_energy.total_mmbtu',
                       'saving_cbill.total_usd',
                       'saving_emission.total_kgCO2e',
                                'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.total_mmbtu'] = results['saving_energy.total_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.total_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum',
                    'sample_weight': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results['saving_energy.total_mmbtu'] = results['saving_energy.total_mmbtu']/results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']/results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']/results['sample_weight']
    results = results.sort_values('saving_cbill.total_usd')
    
    print('sample_weight')
    print(results['sample_weight'])
  
    fig = make_subplots(rows=1, cols=4, horizontal_spacing=0.1, column_widths=[0.02, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.total_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.total_mmbtu'],
        orientation='h'),row=1, col=4)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Emission Savings<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Energy Savings<br>mmbtu", xaxis4_visible=True, yaxis4_visible=False,
                     title='Annual Savings Per Dwelling Unit')
    fig.update_layout(
    autosize=False,
    width=850,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'annual_saving_per_dwelling_unit.svg')

figure_mean_savings(community_results) 

sample_weight
upgrade_name
Mininum Efficiency Whole Home Electrification                      35144.658339
Mininum Efficiency Heat Pump with Electric Heat Backup             39762.000000
Electric Clothes Dryer                                              5914.743706
High Efficiency Heat Pump with Electric Heat Backup                39762.000000
Electric Cooking                                                   12722.230006
Induction Cooking                                                  39329.804348
Heat Pump Clothes Dryer                                            32436.993494
Mininum Efficiency Heat Pump with Existing Heat Backup             37835.613214
Heat Pump Water Heater                                             39724.904866
High Efficiency Whole Home Electrification                         39762.000000
Basic Enclosure                                                    34337.157808
Enhanced Enclosure                                                 35606.210876
Basic Enclosu

# 2. Results for Segments

## Number of buildings for each segment

In [22]:
community_results_baseline = community_results.loc[community_results['upgrade_name'] == 'baseline']
segment1 = community_results_baseline.loc[community_results_baseline['segment'] == 'segment1']
print(segment1['sample_weight'].sum())
segment2 = community_results_baseline.loc[community_results_baseline['segment'] == 'segment2']
print(segment2['sample_weight'].sum())
segment3 = community_results_baseline.loc[community_results_baseline['segment'] == 'segment3']
print(segment3['sample_weight'].sum())
segment4 = community_results_baseline.loc[community_results_baseline['segment'] == 'segment4']
print(segment4['sample_weight'].sum())

1228.1559782608697
2821.2535657706135
48.02173913043478
305.9793706930451


In [23]:
print("Checking sample size in baseline:")
print(pd.concat([
    community_results.loc[community_results["upgrade_name"]=="baseline"].groupby(["segment"])["sample_weight"].count().rename("n_samples"),
    community_results.loc[community_results["upgrade_name"]=="baseline"].groupby(["segment"])["sample_weight"].sum().rename("n_represented"),
], axis=1))

print("\nChecking sample size by upgrade:")
community_results.groupby(["segment", "upgrade_name"])["sample_weight"].count().to_frame().unstack()

Checking sample size in baseline:
          n_samples  n_represented
segment                           
segment1         16    1228.155978
segment2         62    2821.253566
segment3          1      48.021739
segment4         10     305.979371

Checking sample size by upgrade:


sample_weight  \
upgrade_name Basic Enclosure   
segment                        
segment1                16.0   
segment2                40.0   
segment3                 1.0   
segment4                 9.0   

                                                                          \
upgrade_name Basic Enclosure + HPWH + High Efficiency HP/Electric Backup   
segment                                                                    
segment1                                                   16.0            
segment2                                                   62.0            
segment3                                                    1.0            
segment4                                                   10.0            

                                                                           \
upgrade_name Basic Enclosure + High Efficiency Whole Home Electrification   
segment                                                                     
segment1                                                   16.0             
segment2                                                   62.0             
segment3                                                    1.0             
segment4                                                   10.0             

                                                                         \
upgrade_name Electric Clothes Dryer Electric Cooking Enhanced Enclosure   
segment                                                                   
segment1                        NaN              4.0               16.0   
segment2                        2.0             10.0               46.0   
segment3                        NaN              1.0                1.0   
segment4                        NaN              2.0               10.0   

                                                                             \
upgrade_name Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup   
segment                                                                       
segment1                                                   16.0               
segment2                                                   62.0               
segment3                                                    1.0               
segment4                                                   10.0               

                                                                              \
upgrade_name Enhanced Enclosure + High Efficiency Whole Home Electrification   
segment                                                                        
segment1                                                   16.0                
segment2                                                   62.0                
segment3                                                    1.0                
segment4                                                   10.0                

                                                             \
upgrade_name Heat Pump Clothes Dryer Heat Pump Water Heater   
segment                                                       
segment1                        12.0                   16.0   
segment2                        19.0                   62.0   
segment3                         1.0                    1.0   
segment4                        10.0                   10.0   

                                                                  \
upgrade_name High Efficiency Heat Pump with Electric Heat Backup   
segment                                                            
segment1                                                   16.0    
segment2                                                   62.0    
segment3                                                    1.0    
segment4                                                   10.0    

                                                                           \
upgrade_name High Efficiency Whole Home Electrification Induction Cooking   
seg

## 2.1 Upgrade Cost Per Dwelling Unit 

In [24]:
def figure_mean_upgrade_cost(commnity_results):
    results = commnity_results[['upgrade_name',
                       'upgrade_cost_usd',
                                'sample_weight',
                                'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    
    agg_functions = {'upgrade_name': 'first',
                     'segment': 'first',
                     'upgrade_cost_usd': 'sum',
                    'sample_weight': 'sum'}
    
    segment1 = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2 = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3 = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4 = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    
    segment1['upgrade_cost_usd'] = segment1['upgrade_cost_usd']/segment1['sample_weight']
    segment2['upgrade_cost_usd'] = segment2['upgrade_cost_usd']/segment2['sample_weight']
    segment3['upgrade_cost_usd'] = segment3['upgrade_cost_usd']/segment3['sample_weight']
    segment4['upgrade_cost_usd'] = segment4['upgrade_cost_usd']/segment4['sample_weight']
    
    segment1 = segment1.sort_values('upgrade_cost_usd')
    segment2 = segment2.sort_values('upgrade_cost_usd')
    segment3 = segment3.sort_values('upgrade_cost_usd')
    segment4 = segment4.sort_values('upgrade_cost_usd')
    
    segment1['upgrade_cost_usd'] = segment1['upgrade_cost_usd'].round(1)
    segment2['upgrade_cost_usd'] = segment2['upgrade_cost_usd'].round(1)
    segment3['upgrade_cost_usd'] = segment3['upgrade_cost_usd'].round(1)
    segment4['upgrade_cost_usd'] = segment4['upgrade_cost_usd'].round(1)
    
    print('segment1: upgrade cost (usd) per dwelling unit')
    print(segment1['upgrade_cost_usd'])
    print('segment2: upgrade cost (usd) per dwelling unit')
    print(segment2['upgrade_cost_usd'])
    print('segment3: upgrade cost (usd) per dwelling unit')
    print(segment3['upgrade_cost_usd'])
    print('segment4: upgrade cost (usd) per dwelling unit')
    print(segment4['upgrade_cost_usd'])

figure_mean_upgrade_cost(community_results) 

segment1: upgrade cost (usd) per dwelling unit
upgrade_name
Basic Enclosure                                                      828.2
Electric Cooking                                                    1127.3
Induction Cooking                                                   1953.1
Enhanced Enclosure                                                  2458.7
Heat Pump Clothes Dryer                                             2547.5
Heat Pump Water Heater                                              2905.1
High Efficiency Heat Pump with Electric Heat Backup                23617.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        26149.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     27683.6
Basic Enclosure + High Efficiency Whole Home Electrification       30000.7
High Efficiency Whole Home Electrification                         30374.7
Enhanced Enclosure + High Efficiency Whole Home Electrification    31535.3
Mininum Efficiency Heat Pump with Electr

 ## 2.2 Annual Savings Per Dollar Invested

In [25]:
def figure_most_impact_package_savings_per_dollar(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'upgrade_cost_usd', 'sample_weight', 'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results[indicator] = results[indicator]*results['sample_weight']
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                     'upgrade_cost_usd': 'sum',
                     'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4_agg = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg[indicator] = segment1_agg[indicator]/segment1_agg['upgrade_cost_usd']
    segment2_agg[indicator] = segment2_agg[indicator]/segment2_agg['upgrade_cost_usd']
    segment3_agg[indicator] = segment3_agg[indicator]/segment3_agg['upgrade_cost_usd']
    segment4_agg[indicator] = segment4_agg[indicator]/segment4_agg['upgrade_cost_usd']
    
    segment1_agg = segment1_agg.sort_values([indicator], ascending=False)
    segment2_agg = segment2_agg.sort_values([indicator], ascending=False)
    segment3_agg = segment3_agg.sort_values([indicator], ascending=False)
    segment4_agg = segment4_agg.sort_values([indicator], ascending=False)
    
    segment1_agg[indicator] = segment1_agg[indicator].round(5)
    segment2_agg[indicator] = segment2_agg[indicator].round(5)
    segment3_agg[indicator] = segment3_agg[indicator].round(5)
    segment4_agg[indicator] = segment4_agg[indicator].round(5)

    print (indicator)
    print('segment1')
    print(segment1_agg[indicator])
    print('segment2')
    print(segment2_agg[indicator])
    print('segment3')
    print(segment3_agg[indicator])
    print('segment4')
    print(segment4_agg[indicator])
    
    #print (indicator)
    #print('segment1')
    #print (segment1_agg.loc[segment1_agg[indicator].idxmax()]['upgrade_name'])
    #print (segment1_agg.loc[segment1_agg[indicator].idxmax()][indicator])
    #print('segment2')
    #print (segment2_agg.loc[segment2_agg[indicator].idxmax()]['upgrade_name'])
    #print (segment2_agg.loc[segment2_agg[indicator].idxmax()][indicator])
    #print('segment3')
    #print (segment3_agg.loc[segment3_agg[indicator].idxmax()]['upgrade_name'])
    #print (segment3_agg.loc[segment3_agg[indicator].idxmax()][indicator])
    #print('segment4')
    #print (segment4_agg.loc[segment4_agg[indicator].idxmax()]['upgrade_name'])
    #print (segment4_agg.loc[segment4_agg[indicator].idxmax()][indicator])
    
indicator ='saving_emission.total_kgCO2e' # 'saving_cbill.total_usd','saving_energy.total_mmbtu', 'saving_emission.total_kgCO2e'
figure_most_impact_package_savings_per_dollar(indicator, community_results) 

saving_emission.total_kgCO2e
segment1
upgrade_name
Basic Enclosure                                                    0.86086
Enhanced Enclosure                                                 0.37614
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.13871
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.13481
Basic Enclosure + High Efficiency Whole Home Electrification       0.12310
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.12044
High Efficiency Heat Pump with Electric Heat Backup                0.11751
Electric Cooking                                                   0.11674
High Efficiency Whole Home Electrification                         0.11165
Heat Pump Water Heater                                             0.09882
Mininum Efficiency Whole Home Electrification                      0.07668
Mininum Efficiency Heat Pump with Electric Heat Backup             0.06279
Mininum Efficiency Heat Pump with Existing Heat B

## 2.3 Annual Total Savings

In [26]:
def figure_most_impact_package_total_savings(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'sample_weight', 'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results[indicator] = results[indicator]*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                    'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4_agg = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg = segment1_agg.sort_values([indicator], ascending=False)
    segment2_agg = segment2_agg.sort_values([indicator], ascending=False)
    segment3_agg = segment3_agg.sort_values([indicator], ascending=False)
    segment4_agg = segment4_agg.sort_values([indicator], ascending=False)
    
    segment1_agg[indicator] = segment1_agg[indicator].round(1)
    segment2_agg[indicator] = segment2_agg[indicator].round(1)
    segment3_agg[indicator] = segment3_agg[indicator].round(1)
    segment4_agg[indicator] = segment4_agg[indicator].round(1)

    print (indicator)
    print('segment1')
    print(segment1_agg[indicator])
    print('segment2')
    print(segment2_agg[indicator])
    print('segment3')
    print(segment3_agg[indicator])
    print('segment4')
    print(segment4_agg[indicator])

    #print (indicator)
    #print('segment1')
    #print (segment1_agg.loc[segment1_agg[indicator].idxmax()]['upgrade_name'])
    ##print (segment1_agg.loc[segment1_agg[indicator].idxmax()][indicator])
    #print('segment2')
    #print (segment2_agg.loc[segment2_agg[indicator].idxmax()]['upgrade_name'])
    #print (segment2_agg.loc[segment2_agg[indicator].idxmax()][indicator])
    #print('segment3')
    #print (segment3_agg.loc[segment3_agg[indicator].idxmax()]['upgrade_name'])
    #print (segment3_agg.loc[segment3_agg[indicator].idxmax()][indicator])
    #print('segment4')
    #print (segment4_agg.loc[segment4_agg[indicator].idxmax()]['upgrade_name'])
    #print (segment4_agg.loc[segment4_agg[indicator].idxmax()][indicator])

    
indicator ='saving_emission.total_kgCO2e' # 'saving_cbill.total_usd', 'saving_energy.total_mmbtu','saving_emission.total_kgCO2e'
figure_most_impact_package_total_savings(indicator, community_results) 

saving_emission.total_kgCO2e
segment1
upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    4664583.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     4583676.3
Basic Enclosure + High Efficiency Whole Home Electrification       4535551.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        4454644.4
High Efficiency Whole Home Electrification                         4165074.2
High Efficiency Heat Pump with Electric Heat Backup                3408567.7
Mininum Efficiency Whole Home Electrification                      3276294.6
Mininum Efficiency Heat Pump with Electric Heat Backup             2789868.9
Mininum Efficiency Heat Pump with Existing Heat Backup             1848740.6
Enhanced Enclosure                                                 1135807.3
Basic Enclosure                                                     875600.1
Heat Pump Water Heater                                              352573.1
Induction Cooking        

## 2.4 Mean Savings

In [27]:
def figure_most_impact_package_mean_savings(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'sample_weight', 'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results[indicator] = results[indicator]*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                    'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4_agg = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg[indicator] = segment1_agg[indicator]/segment1_agg['sample_weight']
    segment2_agg[indicator] = segment2_agg[indicator]/segment2_agg['sample_weight']
    segment3_agg[indicator] = segment3_agg[indicator]/segment3_agg['sample_weight']
    segment4_agg[indicator] = segment4_agg[indicator]/segment4_agg['sample_weight']
    
    segment1_agg = segment1_agg.sort_values([indicator], ascending=False)
    segment2_agg = segment2_agg.sort_values([indicator], ascending=False)
    segment3_agg = segment3_agg.sort_values([indicator], ascending=False)
    segment4_agg = segment4_agg.sort_values([indicator], ascending=False)
    
    segment1_agg[indicator] = segment1_agg[indicator].round(1)
    segment2_agg[indicator] = segment2_agg[indicator].round(1)
    segment3_agg[indicator] = segment3_agg[indicator].round(1)
    segment4_agg[indicator] = segment4_agg[indicator].round(1)

    print (indicator)
    print('segment1')
    print(segment1_agg[indicator])
    print('segment2')
    print(segment2_agg[indicator])
    print('segment3')
    print(segment3_agg[indicator])
    print('segment4')
    print(segment4_agg[indicator])

    #print (indicator)
    #print('segment1')
    #print (segment1_agg.loc[segment1_agg[indicator].idxmax()]['upgrade_name'])
    #print (segment1_agg.loc[segment1_agg[indicator].idxmax()][indicator])
    #print('segment2')
    #print (segment2_agg.loc[segment2_agg[indicator].idxmax()]['upgrade_name'])
    #print (segment2_agg.loc[segment2_agg[indicator].idxmax()][indicator])
    #print('segment3')
    #print (segment3_agg.loc[segment3_agg[indicator].idxmax()]['upgrade_name'])
    #print (segment3_agg.loc[segment3_agg[indicator].idxmax()][indicator])
    #print('segment4')
    #print (segment4_agg.loc[segment4_agg[indicator].idxmax()]['upgrade_name'])
    #print (segment4_agg.loc[segment4_agg[indicator].idxmax()][indicator])

    
indicator = 'saving_emission.total_kgCO2e'#'saving_cbill.total_usd', 'saving_energy.total_mmbtu', 'pct_saving_energy.total_%', 'saving_emission.total_kgCO2e' 'post-upgrade_energy_burden_2023_cbills.%'
figure_most_impact_package_mean_savings(indicator, community_results) 

saving_emission.total_kgCO2e
segment1
upgrade_name
Mininum Efficiency Whole Home Electrification                      3838.3
Enhanced Enclosure + High Efficiency Whole Home Electrification    3798.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     3732.2
Basic Enclosure + High Efficiency Whole Home Electrification       3693.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        3627.1
High Efficiency Whole Home Electrification                         3391.3
High Efficiency Heat Pump with Electric Heat Backup                2775.4
Mininum Efficiency Heat Pump with Electric Heat Backup             2271.6
Mininum Efficiency Heat Pump with Existing Heat Backup             1505.3
Enhanced Enclosure                                                  924.8
Basic Enclosure                                                     712.9
Heat Pump Water Heater                                              287.1
Electric Cooking                                             

## 4. Most impactful retrofit package for each segment: Pecent stock with net positive bill savings

In [28]:
def figure_positive_negative_savings(segment, commnity_results):
    results = commnity_results[['upgrade_name',
                                  'saving_energy.total_mmbtu',
                                  'saving_cbill.total_usd',
                                  'saving_emission.total_kgCO2e',
                                'sample_weight',
                               'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results = results.loc[results['segment'] == segment]
    results = results.groupby(results['upgrade_name'])
  
    def pos(col): 
      return col[col > 0].count()
  
    def neg(col): 
      return col[col < 0].count() 

    percentage = pd.DataFrame()

    bill = results['saving_cbill.total_usd'].agg([('negative_values', neg),('positive_values', pos)])
    emission = results['saving_emission.total_kgCO2e'].agg([('negative_values', neg),('positive_values', pos)])
    energy = results['saving_energy.total_mmbtu'].agg([('negative_values', neg),('positive_values', pos)])
    
    percentage ['cbill_n'] = bill['negative_values']/(bill['negative_values']+bill['positive_values'])
    percentage ['cbill_p'] = bill['positive_values']/(bill['negative_values']+bill['positive_values'])
    percentage ['emission_n'] = emission['negative_values']/(emission['negative_values']+emission['positive_values'])
    percentage ['emission_p'] = emission['positive_values']/(emission['negative_values']+emission['positive_values'])
    percentage ['energy_n'] = energy['negative_values']/(energy['negative_values']+energy['positive_values'])
    percentage ['energy_p'] = energy['positive_values']/(energy['negative_values']+energy['positive_values'])
    
    
    percentage = percentage.sort_values(('cbill_p'))
    upgrade_name = percentage.index
    
    fig = make_subplots(rows=1, cols=3)
    marker1= dict(color='red')
    marker2= dict(color='green')
    
    fig.add_bar(y=upgrade_name, x=percentage ['cbill_n'], text=percentage ['cbill_n'],
                orientation='h',name = "Negative",row=1, col=1, marker=marker1)
    fig.add_bar(y=upgrade_name, x=percentage ['cbill_p'],text=percentage ['cbill_p'],
                orientation='h', name = "Positive", row=1, col=1, marker=marker2)
    
    fig.add_bar(y=upgrade_name, x=percentage ['emission_n'], text=percentage ['emission_n'],
                orientation='h',showlegend=False, row=1, col=2, marker=marker1)
    fig.add_bar(y=upgrade_name, x=percentage ['emission_p'],text=percentage ['emission_p'],
                orientation='h', showlegend=False, row=1, col=2, marker=marker2)
    
    fig.add_bar(y=upgrade_name, x=percentage ['energy_n'],text=percentage ['energy_n'],
                orientation='h',showlegend=False, row=1, col=3, marker=marker1)
    fig.add_bar(y=upgrade_name, x=percentage ['energy_p'],text=percentage ['energy_p'],
                orientation='h', showlegend=False, row=1, col=3, marker=marker2)
   
  
    fig.update_layout(barmode="relative")
    fig.update_traces(texttemplate='%{text:.0%}', textposition='inside')
    
    fig.update_layout(xaxis_title="Bill", xaxis_visible=True, yaxis_visible=True,
                      xaxis2_title="Emission", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Total Energy", xaxis3_visible=True, yaxis3_visible=False,
                      font=dict(size=10))
    
    fig.update_layout(
        title='Percentage of Household in Positive and Negative Savings')
    
    fig.update_layout(
    autosize=False,
    width=1000,
    height=500,
    font=dict(size=10))
    fig.layout.xaxis1.tickformat = ',.0%'
    fig.layout.xaxis2.tickformat = ',.0%'
    fig.layout.xaxis3.tickformat = ',.0%'
    
    fig.show()
    fig.write_image(fig_dir / f'percentage_positive_saving_{segment}.svg')

segment = 'segment4'
figure_positive_negative_savings(segment, community_results)     


## 5. Most impactful retrofit package for each segment: energy burden

In [29]:
def figure_most_impact_package_mean_energy_burden(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'baseline_energy_burden_2023_cbills.%', 'sample_weight', 'segment']]
    results.replace([np.inf, -np.inf], np.nan, inplace=True)
    results.dropna(subset=['baseline_energy_burden_2023_cbills.%'], how="all", inplace=True)
    results.to_csv('test.csv')
    results[indicator] = results[indicator]*results['sample_weight']
    results['baseline_energy_burden_2023_cbills.%'] = results['baseline_energy_burden_2023_cbills.%']*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                     'baseline_energy_burden_2023_cbills.%': 'sum',
                    'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4_agg = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg['baseline_energy_burden_2023_cbills.%'] = segment1_agg['baseline_energy_burden_2023_cbills.%']/segment1_agg['sample_weight']
    segment2_agg['baseline_energy_burden_2023_cbills.%'] = segment2_agg['baseline_energy_burden_2023_cbills.%']/segment2_agg['sample_weight']
    segment3_agg['baseline_energy_burden_2023_cbills.%'] = segment3_agg['baseline_energy_burden_2023_cbills.%']/segment3_agg['sample_weight']
    segment4_agg['baseline_energy_burden_2023_cbills.%'] = segment4_agg['baseline_energy_burden_2023_cbills.%']/segment4_agg['sample_weight']
    
    segment1_agg[indicator] = segment1_agg[indicator]/segment1_agg['sample_weight']
    segment2_agg[indicator] = segment2_agg[indicator]/segment2_agg['sample_weight']
    segment3_agg[indicator] = segment3_agg[indicator]/segment3_agg['sample_weight']
    segment4_agg[indicator] = segment4_agg[indicator]/segment4_agg['sample_weight']
    
    segment1_agg['energy_burden_diff'] = segment1_agg['baseline_energy_burden_2023_cbills.%'] - segment1_agg[indicator]
    segment2_agg['energy_burden_diff'] = segment2_agg['baseline_energy_burden_2023_cbills.%'] - segment2_agg[indicator]
    segment3_agg['energy_burden_diff'] = segment3_agg['baseline_energy_burden_2023_cbills.%'] - segment3_agg[indicator]
    segment4_agg['energy_burden_diff'] = segment4_agg['baseline_energy_burden_2023_cbills.%'] - segment4_agg[indicator]
    
    segment1_agg = segment1_agg.sort_values(['energy_burden_diff'], ascending=False)
    segment2_agg = segment2_agg.sort_values(['energy_burden_diff'], ascending=False)
    segment3_agg = segment3_agg.sort_values(['energy_burden_diff'], ascending=False)
    segment4_agg = segment4_agg.sort_values(['energy_burden_diff'], ascending=False)
    
    segment1_agg = segment1_agg[segment1_agg.upgrade_name != 'baseline']
    segment2_agg = segment2_agg[segment2_agg.upgrade_name != 'baseline']
    segment3_agg = segment3_agg[segment3_agg.upgrade_name != 'baseline']
    segment4_agg = segment4_agg[segment4_agg.upgrade_name != 'baseline']
    
    segment1_agg = segment1_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    segment2_agg = segment2_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    segment3_agg = segment3_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    segment4_agg = segment4_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    
    segment1_agg['before'] = segment1_agg['before'].round(1)
    segment2_agg['before'] = segment2_agg['before'].round(1)
    segment3_agg['before'] = segment3_agg['before'].round(1)
    segment4_agg['before'] = segment4_agg['before'].round(1)
    segment1_agg['after'] = segment1_agg['after'].round(1)
    segment2_agg['after'] = segment2_agg['after'].round(1)
    segment3_agg['after'] = segment3_agg['after'].round(1)
    segment4_agg['after'] = segment4_agg['after'].round(1)
    
    pd.set_option('display.max_colwidth', 3000)
    print ('energy burden (%) before and after upgrade')
    print('segment1')
    print(segment1_agg[['before', 'after']])
    print('segment2')
    print(segment2_agg[['before', 'after']])
    print('segment3')
    print(segment3_agg[['before', 'after']])
    print('segment4')
    print(segment4_agg[['before', 'after']])
  
    #print ('energy burden (%) before and after upgrade')
    #print('segment1')
    #print ('baseline')
    #print (segment1_agg.loc[segment1_agg['energy_burden_diff'].idxmax()]['baseline_energy_burden_2023_cbills.%'])
    #print (segment1_agg.loc[segment1_agg['energy_burden_diff'].idxmax()]['upgrade_name'])
    #print (segment1_agg.loc[segment1_agg['energy_burden_diff'].idxmax()][indicator])
    #print('segment2')
    #print ('baseline')
    #print (segment2_agg.loc[segment2_agg['energy_burden_diff'].idxmax()]['baseline_energy_burden_2023_cbills.%'])
    #print (segment2_agg.loc[segment2_agg['energy_burden_diff'].idxmax()]['upgrade_name'])
    #print (segment2_agg.loc[segment2_agg['energy_burden_diff'].idxmax()][indicator])
    #print('segment3')
    #print ('baseline')
    #print (segment3_agg.loc[segment3_agg['energy_burden_diff'].idxmax()]['baseline_energy_burden_2023_cbills.%'])
    #print (segment3_agg.loc[segment3_agg['energy_burden_diff'].idxmax()]['upgrade_name'])
    #print (segment3_agg.loc[segment3_agg['energy_burden_diff'].idxmax()][indicator])
    #print('segment4')
    #print ('baseline')
    #print (segment4_agg.loc[segment4_agg['energy_burden_diff'].idxmax()]['baseline_energy_burden_2023_cbills.%'])
    #print (segment4_agg.loc[segment4_agg['energy_burden_diff'].idxmax()]['upgrade_name'])
    #print (segment4_agg.loc[segment4_agg['energy_burden_diff'].idxmax()][indicator])

figure_most_impact_package_mean_energy_burden('post-upgrade_energy_burden_2023_cbills.%', community_results) 

energy burden (%) before and after upgrade
segment1
                                                                 before  after
upgrade_name                                                                  
Enhanced Enclosure + High Efficiency Whole Home Electrification     8.0    4.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup      8.0    4.9
Basic Enclosure + High Efficiency Whole Home Electrification        8.0    5.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup         8.0    5.5
High Efficiency Whole Home Electrification                          8.0    6.2
Enhanced Enclosure                                                  8.0    6.8
High Efficiency Heat Pump with Electric Heat Backup                 8.0    7.0
Basic Enclosure                                                     8.0    7.1
Mininum Efficiency Heat Pump with Existing Heat Backup              8.0    7.6
Heat Pump Water Heater                                              8.0    7.7


/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_30393/175440032.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_30393/175440032.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_30393/175440032.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/